---
title: "Recursion Optimizations"
author: "Vahram Poghosyan"
date: "2023-01-13"
categories: ["Recursion", "Functional Programming"]
format:
  html:
    toc: true
    toc-depth: 5
    code-fold: true
jupyter: python3
include-after-body:
  text: |
    <script type="application/javascript" src="../../javascript/light-dark.js"></script>
---

# Recursion Optimizations

Recursive algorithms, while elegant and expressive, can sometimes lead to performance issues due to the overhead of function calls and potential for repeated computations. Several optimization techniques can be employed to enhance their efficiency. One common method is **memoization**, which stores the results of expensive function calls and reuses them when the same inputs occur again, avoiding repeated computations. Another technique is **tail recursion optimization**, where the recursive call is the final operation in the function, allowing the system to reuse the current stack frame for each recursive call. Tail recursion optimization reduces the space complexity from $O(n)$ to $O(1)$. Additionally, **iterative solutions** can often be more efficient than their recursive counterparts, so converting a recursive algorithm to an iterative one can be considered an optimization. Understanding these techniques, and variations on them, can greatly improve the performance of our recursive algorithms.

#### Tail Recursion - Avoiding Stack Overflow

**Stack overflow** (which is when the system runs out of short term memory) is a common concern when working with recursive functions or when doing [functional programming](../functional_programming/functional_programming.ipynb), where function composition is the mode in which we think. **Tail recursion optimization** helps us drastically cut the number of stack frames and, as mentioned earlier, makes our recursive algorithms take a constant amount of space instead of a linear amount (or worse).


##### Classic Example: Factorial

Let's take the classic example of calculating a factorial. 

**Naive Recursive Implementation:**

In [2]:
#| code-fold: false
def factorial(n):
    if n == 0: # Base case: 0! = 1
        return 1
    else: 
        return n * factorial(n-1) # Recursive step

factorial(4)

24

From the below untangled definition of factorial (for $n=4$) we can surmise what goes on in the stack. The stack first fills up with stack frames for `factorial(n)` down to `factorial(0)`, which is the last frame on the stack before it begins to pop and actual evaluation happens. 

$$
\begin{equation}
    \begin{split}
        factorial(4) & = 4 * factorial(3) \\
        & = 4 * (3 * factorial(2)) \\
        & = 4 * (3 * (2 * factorial(1))) \\
        & = 4 * (3 * (2 * (1 * factorial(0)))) \\
        & = 4 * (3 * (2 * (1 * 1))) \\ 
        & = 4 * (3 * (2 * 1)) \\
        & = 4 * (3 * 2) \\
        & = 4 * 6 \\
        & = 24 
    \end{split}
\end{equation}
$$

As we can see the function is called for $n = 4$ down to the base case of $n = 0$ (each call stacking up in memory) before evaluation even begins. Evaluation then happens step-by-step inside each stack frame until all of them have popped. 

It's not immediately clear how to make the calls independent of each other given that there is a multiplicative factor in front of the recursive call (which is what makes this particular function *fail* to be tail-recursive). It helps to think in terms of *carried state*. In this case the idea is simple, if we can carry the state of the current stack frame into the next one as input, then we can pop each frame right after it calls the next frame. Why? Because at that point, having carried its state into the next frame, the current frame exhausts its usefulness.

In the case of the factorial function above, this means that in the tail-recursive implementation the stack is not filled up with as many frames of recursive `factorial` calls as the input ($n$) is big. There are still $n$ total calls, however the memory used in the stack is held constant (at a single frame in this case) as each old frame gives way to the new one. 

So, for now, let's *define* a magic function called `go(n,acc)` with inputs `n` and what's called an *accumulator* `acc` such that `factorial(n) := go(n,1)`. We take this to be *by construction*. The function `go` will be the *tail-recursive helper* of `factorial`. The accumulator `acc`, which is initialized to `1`, will be used to remember the state inside the current stack frame (in this case just the multiplicative factor before the recursive call). 

But so far we've only given `go(n, acc)` its desired properties without actually defining it. The following is the tail-recursive version of factorial which includes the definition of `go`. 

**Tail-Recursive Factorial:**

In [16]:
#| code-fold: false
def factorial(n):
    def go(n,acc): # Helper function with an accumulator
        if n == 0: # Base case: 0! = 1
            return acc
        else:
            return go(n-1, n * acc) # Tail-recusrive step
        
    return go(n,1) # Delegate the problem solution to a helper function

factorial(4)

24


Let's unpack this: 

$$
\begin{equation}
    \begin{split}
        factorial(4) & = go(4,1) \\
        & = go(3,4) \\
        & = go(2,12) \\
        & = go(1,24) \\
        & = go(0,24) \\ 
        & = 24
    \end{split}
\end{equation}
$$

Right away we can see that, with this approach, we can pop the previous stack frame at any time without losing any information it holds because all state is carried over from the previous frame into the current one by the accumulator and, finally, returned at the end. A visual cue of this fact is that in the expression above evaluation happens immediately, rather than step-by-step (with each step corresponding to the popping of a stack frame), as is the case in the naive implementation.

::: {.callout-tip title="Note" appearance="minimal" collapse="false"}
It's important to note that this effort only pays off if the language compiler in question supports **TCO** (**Tail Call Optimization**). Most, in fact, do. If the language supports TCO the compiler can recognize tail calls and simply pop the current stack frame after the recursive call, replacing it with the subsequent call (rather than blindly stacking frames on top of each other as in the naive recursive algorithm)
:::